In [1]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, Reshape
from keras.layers import Conv1D, MaxPooling1D
from keras.optimizers import SGD, Adagrad, Adam
from keras.callbacks import EarlyStopping
import read
import numpy as np
import sklearn.metrics
from sklearn.model_selection import train_test_split
from keras.models import model_from_yaml
from keras import backend as K
print K.backend()
import tensorflow

Using TensorFlow backend.


tensorflow


In [2]:
def keras_model():

    model = Sequential()
    
    #model.add(Reshape((n, 4), input_shape=(n*4,)))
    
    model.add(Conv1D(32, 20, padding="same", input_shape=(n, 4,)))
    print model.input_shape
    print model.output_shape
    model.add(Activation('relu'))
    print model.output_shape
    model.add(MaxPooling1D(padding="same", strides=None, pool_size=200))

    # after Max Pooling
    print model.output_shape

    model.add(Flatten())
    print model.output_shape
    model.add(Dense(512))

    # Flattened 
    print model.output_shape

    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1))

    # Finally
    print model.output_shape

    model.add(Activation('sigmoid'))
    return model

In [3]:
import cv2

In [7]:
def train_and_test(X_train, y_train, X_test, y_test, n):

    model = keras_model()
    adam = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
    adagrad = Adagrad(lr=0.05, epsilon=1e-08, decay=0.0)
    sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(loss='binary_crossentropy', optimizer=sgd, metrics=['accuracy'])
    early_stopping = EarlyStopping(monitor='val_loss', patience=3, verbose=1)
    hist = model.fit(X_train, y_train, validation_split=0.1, callbacks=[early_stopping], batch_size= 200, nb_epoch= 20)
    #hist = model.fit(X_train, y_train, validation_split=0.1, batch_size= 200, shuffle = False, nb_epoch= 10)
    print hist.history
    probas = model.predict_on_batch(X_test_rs)
    print sklearn.metrics.average_precision_score(y_test, probas)
    np.savetxt("probas", probas)
    print "Done"
    return model

In [4]:
n = 200
X,y = read.get_matrix("shared.genomicnegs.txt")
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print "Done"

Done


In [5]:
K.image_data_format()
X_train_rs = np.reshape(X_train, (-1,200,4))
X_test_rs = np.reshape(X_test, (-1,200,4))

In [8]:
print X_train
print y_train

print "Start"
model = train_and_test(X_train_rs, y_train, X_test_rs, y_test, n)

print "Done"

[[0 1 0 ..., 0 0 1]
 [0 1 0 ..., 1 0 0]
 [0 0 1 ..., 0 1 0]
 ..., 
 [0 0 0 ..., 1 0 0]
 [0 1 0 ..., 1 0 0]
 [0 1 0 ..., 1 0 0]]
[0 0 1 ..., 0 1 1]
Start
(None, 200, 4)
(None, 200, 32)
(None, 200, 32)
(None, 1, 32)
(None, 32)
(None, 512)
(None, 1)
Train on 59482 samples, validate on 6610 samples
Epoch 1/20
59482/59482 [==============================] - 8s - loss: 0.6652 - acc: 0.5887 - val_loss: 0.6340 - val_acc: 0.6346
Epoch 2/20
59482/59482 [==============================] - 8s - loss: 0.5283 - acc: 0.7373 - val_loss: 0.4004 - val_acc: 0.8340
Epoch 3/20
59482/59482 [==============================] - 7s - loss: 0.3501 - acc: 0.8499 - val_loss: 0.3320 - val_acc: 0.8514
Epoch 4/20
59482/59482 [==============================] - 7s - loss: 0.3126 - acc: 0.8650 - val_loss: 0.3331 - val_acc: 0.8520
Epoch 5/20
59482/59482 [==============================] - 7s - loss: 0.2911 - acc: 0.8763 - val_loss: 0.2994 - val_acc: 0.8723
Epoch 6/20
59482/59482 [==============================] - 8s - loss: 

In [9]:
# Trained
print "trained miodel"
print model
print model.layers[-1].get_config()


trained miodel
{'activation': 'sigmoid', 'trainable': True, 'name': 'activation_6'}


In [10]:
## Using the fucntional API. This method returns a function to find saliency given input and learning phase. 
def compile_saliency_function(model):
    
    inp = model.layers[0].input
    outp = model.layers[-1].output
    print inp
    print outp
    max_output = K.max(outp, axis=1)
    print max_output
    saliency = K.gradients(K.sum(max_output), inp)[0]
    print saliency
    return K.function([inp, K.learning_phase()], [saliency])

In [11]:
## saliency_fn on the given model. 
saliency_fn = compile_saliency_function(model)

Tensor("conv1d_2_input:0", shape=(?, 200, 4), dtype=float32)
Tensor("activation_6/Sigmoid:0", shape=(?, 1), dtype=float32)
Tensor("Max:0", shape=(?,), dtype=float32)
Tensor("gradients_2/conv1d_2/convolution/ExpandDims_grad/Reshape:0", shape=(?, 200, 4), dtype=float32)


In [12]:
## Computing Saliency. 
n = 200
test_seq,_= read.get_matrix("class.B.txt")
test_seq_rs = np.reshape(test_seq, (-1,200,4))

arr_sal = saliency_fn([X_train_rs[:10], 0])
outp = (np.asarray(arr_sal))[0][8]
print outp.shape

ta = np.transpose(outp)
print ta.shape
np.savetxt("check_saliency",ta)

# Need to do a grad * input.

(200, 4)
(4, 200)


In [70]:
## Interpreting Fitlers.
W,b = model.layers[0].get_weights()
print W.shape
print b.shape

# dim1 = filter length
# dim2 = A,T,G,C
# dim3 = filter number


def get_filter(nb, W):
    A = list()
    T = list()
    G = list()
    C = list()
    for position in range(20):
        A.append(W[position][0][nb])
        T.append(W[position][1][nb])
        G.append(W[position][2][nb])
        C.append(W[position][3][nb])
        filter_cnn = np.vstack((A,T,G,C))
        # So these are the weights of the 4 channels for the first filter
        np.savetxt("filter" + str(nb), filter_cnn)

for i in range(64):
    get_filter(i,W)

(20, 4, 64)
(64,)


In [ ]:
# Serialising model to yaml
yamlstr = model.to_yaml()

In [26]:
# Saving model as yaml file. (Switch to json. Was using yaml for deeplift compatibility but that doesn't work 
# for me yet anyway. )
with open("CTCF_model.yaml", "w") as yaml_file:
    yaml_file.write(yamlstr)
print "Done"

Done
